# Trace visual calculations in a report and extract definitions
This notebook extracts visual calculations from an existing report. The following steps are applied: 
- Get the report definition (base64)
- Converts base64 to json 
- Identify visual containing visual calculations 
- Extract visual calculation definition(s) from each visual
- Print visual calculation name, language and expression to dataframe


<mark>**Disclaimer:** that this notebook contains AI generated code</mark>

### Importing libraries and defining variables

In [1]:
# Importing semantic link labs packages
%pip install semantic-link-labs
import sempy_labs as labs
import sempy_labs.report as rep
from sempy_labs.report import ReportWrapper
import json
import base64
import pandas as pd

StatementMeta(, 44699461-f127-49da-8579-fda7098a6699, 8, Finished, Available, Finished)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 81.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: semantic-link-sempy
    Found existing installation: semantic-link-sempy 0.10.2
    Not uninstalling semantic-link-sempy at /home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages, outside environment /nfs4/pyenv-1352ae36-a456-4135-ad8b-71e048d6f4e4
    Can't uninstall 'semantic-link-sempy'. No files were found to uninstall.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [2]:
# Define variables
report_name = "Contoso Sales" # Enter the report name
report_workspace = "Visual Calculations book" # Enter the workspace in which the report exists
rpt = ReportWrapper(report=report_name, workspace=report_workspace) # Setting up the report wrapper to be used later

StatementMeta(, 44699461-f127-49da-8579-fda7098a6699, 10, Finished, Available, Finished)

### Get report definition

In [3]:
# This step extracts the base64 payload for the different report elements
# if you want to reuse report elements, this can be useful, though we cannot directly extract the visual calculations
reportdef = labs.report.get_report_definition(
    report=report_name, 
    workspace=report_workspace , # defaults to the current workspace, 
    return_dataframe= True # if True, returns a dataframe. If False, returns a json dictionary.
    )

StatementMeta(, 44699461-f127-49da-8579-fda7098a6699, 11, Finished, Available, Finished)

In [6]:
# Extract and decode the JSON payload from Base64
row = reportdef[reportdef["path"] == "report.json"].iloc[0]
base64_payload = row["payload"]
json_bytes = base64.b64decode(base64_payload)
json_str = json_bytes.decode("utf-8")
json_obj = json.loads(json_str)

StatementMeta(, 44699461-f127-49da-8579-fda7098a6699, 14, Finished, Available, Finished)

### Extracting visuals

In [5]:
# Recursive function to find all NativeVisualCalculation objects anywhere in a nested JSON
def find_native_visual_calculations(obj):
    results = []
    if isinstance(obj, dict):
        for k, v in obj.items():
            if k == "NativeVisualCalculation" and isinstance(v, dict):
                results.append(v)
            else:
                results.extend(find_native_visual_calculations(v))
    elif isinstance(obj, list):
        for item in obj:
            results.extend(find_native_visual_calculations(item))
    return results

StatementMeta(, 44699461-f127-49da-8579-fda7098a6699, 13, Finished, Available, Finished)

In [14]:
# trying to find the rows in the json which contain a definition like visual calculations
def find_strings_with_substring(obj, substring="NativeVisualCalculation"):
    found_strings = []
    if isinstance(obj, dict):
        for v in obj.values():
            found_strings.extend(find_strings_with_substring(v, substring))
    elif isinstance(obj, list):
        for item in obj:
            found_strings.extend(find_strings_with_substring(item, substring))
    elif isinstance(obj, str):
        if substring in obj:
            found_strings.append(obj)
    return found_strings

# Search all strings in json_obj that contain the key substring
visuals_with_vc = find_strings_with_substring(json_obj, "NativeVisualCalculation")

print(f"Found {len(visuals_with_vc)} visuals containing Visual Calculation in {report_name}")


StatementMeta(, 44699461-f127-49da-8579-fda7098a6699, 22, Finished, Available, Finished)

Found 45 visuals containing Visual Calculation in Contoso Sales


### Extracting visual calculations from visuals

In [16]:
# as a line may contain multiple visual calculations, extracting the number of visual calculations from the strings
all_nvc_objects = []

for s in visuals_with_vc:
    try:
        # Sometimes the string might be JSON escaped (like config fields)
        parsed = json.loads(s)
    except Exception:
        # Not JSON, skip or continue
        continue
    
    # Now find native visual calculations inside this parsed JSON snippet
    nvc_objs = find_native_visual_calculations(parsed)
    all_nvc_objects.extend(nvc_objs)

print(f"Extracted {len(all_nvc_objects)} NativeVisualCalculation (nvc) objects from embedded JSON strings")


StatementMeta(, 44699461-f127-49da-8579-fda7098a6699, 24, Finished, Available, Finished)

Extracted 115 NativeVisualCalculation (nvc) objects from embedded JSON strings


In [16]:
# Build list of dictionaries to extract the Language, Expression and Name of the visual calculations
records = []
for nvc in all_nvc_objects:
    language = nvc.get("Language", "").strip()
    expression = nvc.get("Expression", "").replace('\n', ' ').strip()
    name = nvc.get("Name", "").strip()
    records.append({"Name": name, "Language": language, "Expression": expression})

# Create DataFrame
df = pd.DataFrame(records)

# Remove duplicates and reset index
df = df.drop_duplicates().reset_index(drop=True)

# Display with better formatting: no index, wrap text for expressions 
from IPython.display import display
pd.set_option('display.max_colwidth', None)  # show full Expression text without truncation

display(df.style.set_properties(**{
    'text-align': 'left',
    'white-space': 'pre-wrap',
    'word-wrap': 'break-word',
}))

StatementMeta(, 0d5f12b7-5e00-4474-9f60-18f07bc58ff6, 30, Finished, Available, Finished)

,Name,Language,Expression
0,Highlight,dax,"VAR __h = COLLAPSE ( EXPAND ( MAX ( [Sales Amount] ), ROWS ), ROWS ) RETURN IF ( MONTH ( [Date] ) = [MonthSelected Value], __h * 1.2, BLANK () )"
1,Versus next,dax,[Average Product Price] - NEXT([Average Product Price])
2,Versus previous,dax,"[Costs] - PREVIOUS ( [Costs], 2 )"
3,Versus previous OFFSET,dax,"[Costs] - CALCULATE ( [Costs], OFFSET ( -2 ) )"
4,Vs prev cols rows,dax,"[Costs] - PREVIOUS([Costs], COLUMNS ROWS)"
5,Vs prev cols,dax,"[Costs] - PREVIOUS([Costs], COLUMNS)"
6,Versus next,dax,[Average of Price] - NEXT ( [Average of Price] )
7,Versus next in Category,dax,"[Average of Price] - NEXT ( [Average of Price], HIGHESTPARENT )"
8,Versus next in Category OFFSET,dax,"[Average of Price] - CALCULATE ( [Average of Price], OFFSET ( 1, ROWS, HIGHESTPARENT ) )"
9,Rank,dax,RANK(ROWS)
